In [ ]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import win32com.client

In [ ]:
#Open STK, get the IAgSTKObjectRoot
uiApplication = win32com.client.Dispatch("STK12.Application")
uiApplication.Visible=True
stkRoot = uiApplication.Personality2

In [ ]:
#Create a new scenario
stkRoot.NewScenario("Python_AER_Polar_Plot")
scenario = stkRoot.CurrentScenario

In [ ]:
#Add a satellite
satellite = scenario.Children.New(18, "MySatellite") #eSatellite

#Set and get the propagator
satellite.SetPropagatorType(7) #ePropagatorTwoBody)
twoBodyProp = satellite.Propagator

#Get initial state and change the inclination
initialState = twoBodyProp.InitialState.Representation.ConvertTo(1) #eOrbitStateClassical
initialState.Orientation.Inclination = 60

#Assign initial state and propagate
twoBodyProp.InitialState.Representation.Assign(initialState)
twoBodyProp.Propagate()

In [ ]:
#Add a default facility
facility = scenario.Children.New(8, "MyFacility") #eFacility

In [ ]:
#Compute access
access = facility.GetAccessToObject(satellite)
access.ComputeAccess()

In [ ]:
#Get all the access interval start and stop times
accessReport = access.DataProviders.Item("Access Data")
accessIntervals = accessReport.ExecElements(scenario.StartTime, scenario.Stoptime, ["Access Number","Start Time", "Stop Time"])
accessIntervals = list(accessIntervals.DataSets.ToArray())

In [ ]:
#Get Az El data for each access interval
aerData = []
for accessInterval in accessIntervals:
    aerReport = access.DataProviders.GetDataPrvInfoFromPath("AER Data/Default").ExecElements(accessInterval[1], accessInterval[2], 20, ["Azimuth", "Elevation"])
    aerData.append(list(aerReport.DataSets.ToArray()))

In [ ]:
#Plot Az El for each interval on a polar plot with North up.
figure = plt.figure(figsize=(12,7.5))
axes = plt.axes(projection="polar")
axes.set_theta_zero_location("N")
axes.set_theta_direction(-1) #Theta increase clockwise
axes.set_rgrids(range(-90,1,15), labels=["","75"+u"\u00b0","60"+u"\u00b0","45"+u"\u00b0","30"+u"\u00b0","15"+u"\u00b0"], angle=90) #Custom ticks
axes.set_rlim(-90,0) #set hard limits on r values
axes.set_title("Az El Polar Plot MyFacility-MySatellite")
for aer, accessInterval in zip(aerData, accessIntervals):
    aer = np.array(aer)
    az = np.deg2rad(aer[:,0]) #convert azimuth from deg to rad
    el = -aer[:,1] #plot the negative of elevation
    
    axes.plot(az[0],el[0], 'g^', markersize=15, clip_on=False) #mark rise location with green up
    axes.plot(az[1:-1],el[1:-1], label=f"{accessInterval[1][:-10]} - {accessInterval[2][:-10]}", linewidth=4)
    axes.plot(az[-1],el[-1], 'rv', markersize=15, clip_on=False) #mark set location with red down

legend = figure.legend()